## Credit

Notes are taken from NLPlanet Practical NLP with Python course section 2.5 Hugging Face Pipeline for Quick Prototyping
* https://www.nlplanet.org/course-practical-nlp/02-practical-nlp-first-tasks/05-huggingface-pipeline 

Authored by Fabio Chiusano
* https://medium.com/@chiusanofabio94

**All quotes '' are sourced from the NLPlanet course.**

## Pipeline

<u>Pipelines:</u>
* A series of data processing steps executed in a specific order to achieve a result better suited for things like machine learning.
* <u>Pipelines in Hugging Face:</u>
    * Their pipeline library assists in using models for inferencing.
    * Their pipelines are "objects that abstract most of the complex code from the library, offering a simple API dedicated to sever tasks."
        * Tasks include:
            * Named Entity Recognition
            * Masked Language Modeling
            * Sentiment Analysis
            * Feature Extraction
            * Question Answering
        * Source: https://huggingface.co/docs/transformers/main_classes/pipelines

### Constructing a pipeline object

The constructor takes a *task* argument as input. 
Each *task* value has a corresponding pipeline.
The available *task* values are:
* "audio-classification"
* "automatic-speech-recognition"
* "conversational"
* "feature-extraction"
* "fill-mask"
* "image-classification"
* "question-answering"
* "table-question-answering"
* "text2text-generation"
* "text-classification" or alias: "sentiment-analysis"
* "text-generation"
* "token-classification" or alias: "ner"
* "translation"
* "translation_xx_to_yy"
* "summarization"
* "zero-shot-classification"

## Example pipelines:

In [ ]:
# Install transformers library
!pip install transformers

In [1]:
# Import pipeline class
from transformers import pipeline

### Sentiment Analysis Pipeline

In [2]:
# Model by task

model = pipeline(task="sentiment-analysis")
# downloads a model for sentiment analysis
# without specifying a model, the default model for the specified task is downloaded

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [31]:
# Specified model 

model = pipeline(model = "distilbert-base-uncased-finetuned-sst-2-english")

In [34]:
# Use for computation

text = "Happy New Years!"

comp = model(text)
# model performs sentiment-analysis pipeline on the given text

print(f"""
Text:
{text}

Computation:
{comp}
""")


Text:
Happy New Years!

Computation:
[{'label': 'POSITIVE', 'score': 0.9998683929443359}]



In [39]:
# Computing a list of text

text = [
    "I had a great Christmas",
    "I didn't get what I wanted for Christmas"
]
comp = model(text)

for i in range(len(text)):
    print(f"""
    Text:
    {text[i]}
    
    Computation:
    {comp[i]}
    """)


    Text:
    I had a great Christmas
    
    Computation:
    {'label': 'POSITIVE', 'score': 0.9998540878295898}
    

    Text:
    I didn't get what I wanted for Christmas
    
    Computation:
    {'label': 'NEGATIVE', 'score': 0.999136745929718}
    


### Question Answering Pipeline

In [40]:
# Initialize model
model = pipeline(task="question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [45]:
answer = model(
    question = "What date is Christmas?", 
    # pass a question for the model to answer
    context = "Christmas is on Monday, the 25th of December, 2023"
    # provide context for the model to search for the answer to the question in
)
print(f"""
Raw Answer: 
{answer}

Start-End Indicies:
{answer['start']}-{answer['end']}

Answer to Question: 
{answer['answer']}
""")


Raw Answer: 
{'score': 0.4008321166038513, 'start': 16, 'end': 50, 'answer': 'Monday, the 25th of December, 2023'}

Start-End Indicies:
16-50

Answer to Question: 
Monday, the 25th of December, 2023



### Translation Pipeline

In [47]:
# Instantiate translation model
model = pipeline(task="translation_en_to_fr")
# translation_en_to_fr model translates english to french

In [48]:
# Translate a piece of text

text = "Call me Santa"
translation = model(text)
print(translation)

[{'translation_text': 'Appelez-moi Père Nol'}]


### Text Generation Pipeline

In [64]:
# Use the GPT2 model
model = pipeline(model="gpt2")

In [66]:
incipit = "Superman is"
answer = model(incipit)
print(answer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Superman is just one more thing that I don't want to see happen in DC Comics. Even if there wasn't something like the Joker that was really a comic and didn't seem like a bad idea, he wouldn't fit in the DC universe"}]


### Conversation Pipeline

Based on example from https://huggingface.co/microsoft/DialoGPT-medium?text=Hey+my+name+is+Thomas%21+How+are+you%3F

In [97]:
# Imports 

from transformers import AutoModelForCausalLM, AutoTokenizer
# AutoModelForCasualLM class allows the automatic loading of a pre-trained language model for casual language 
# AutoTokenizer class helps tokenize input text
import torch
# Library for manipulating tensors

In [137]:
# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

In [138]:
# Chatting with a model 
# A warning message is unavoidable with this piece of code

for step in range(5):
    # Take and process user input
    user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # tokenizer.encode() converts input text to token IDs
    # input() takes user input
    # tokenizer.eos_token is an end-of-sequence token and marks the end of a sequence/text when tokenizing
    # return_tensors='pt' returns tokens as PyTorch tensor

    # Add input to history
    bot_input_ids = torch.cat([chat_history_ids, user_input_ids], dim=-1) if step > 0 else user_input_ids
    # torch.cat() concatenates tensors
        # takes a sequence (like a list) of tensors and the dimension to concatenate along
        # dimension: dim
            # dim=0 concatenates tensors along its rows (same number of columns, adds more rows)
            # dim=1 concatenates tensors along its columns (same number of rows, adds more columns)
            # dim=-1 concatenates along the last dimension in a tensor (extends only the last dimension)
            # Example: 2x3 and 2x1 tensors along dim 1 -> 2x4 tensor
                # (2 rows, 4 columns per row)
    # if statement sets bot_input_ids to user_input_ids if this is the first message

    # Generate response
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    # model.generate() method generates new sequences of tokens based on provided input of token IDs
        # max_length parameter sets the maximum length of token IDs in the generated sequence
        # pad_token_id specifies the token ID used for padding
            # adds the specified token to the end of sequences to reach a uniform length in each sequence
    # tokenizer.eos_token_id is the token ID of the end_of_sequence token

    # pretty print last output tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    # {} contains the formatted text
    # .format is string formatting
    # tokenizer.decode() converts a sequence of token IDs back into proper text using the tokenizer's vocabulary
        # skip_special_tokens instructs the decoder to skip special tokens such as padding or eos tokens
    # .shape[-1] retrieves the number of tokens of the tensor 
    # chat_history_ids[:, bot_input_ids.shape[-1]:][0]
        # first : selects all rows in the tensor
        # .shape[-1]: selects columns from a specified index (.shap[-1] being an index)
        # [0] accesses the first element of the sliced tensor

>> User: hello


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: Hello! :D


>> User: How are you?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm so sorry!


>> User: You are forgiven.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm so sorry!


#### Using Conversation object:

In [150]:
# Instantiate
model = pipeline("conversational", "microsoft/DialoGPT-medium", pad_token_id=50256)

In [154]:
from transformers import Conversation

conversation = Conversation("Hello!")
# creates Conversation object loaded with the initial user input

conversation = model(conversation)
# updates the conversation object with a generated response

print("DialoGPT>> " + conversation.generated_responses[-1])
# .generated_responses holds a list of the model's reponses
    # [-1] accesses the most recent response

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT>> Hello! :D
